<a href="https://colab.research.google.com/github/bascoul/Ml-Agents/blob/master/Part1_TensorFlow_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="center">
  <td align="center"><a target="_blank" href="http://introtodeeplearning.com">
        <img src="http://introtodeeplearning.com/images/colab/mit.png" style="padding-bottom:5px;" />
      Visit MIT Deep Learning</a></td>
  <td align="center"><a target="_blank" href="https://colab.research.google.com/github/aamini/introtodeeplearning/blob/master/lab1/solutions/Part1_TensorFlow_Solution.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/colab.png?v2.0"  style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/aamini/introtodeeplearning/blob/master/lab1/solutions/Part1_TensorFlow_Solution.ipynb">
        <img src="http://introtodeeplearning.com/images/colab/github.png"  height="70px" style="padding-bottom:5px;"  />View Source on GitHub</a></td>
</table>

# Copyright Information


In [0]:
# Copyright 2020 MIT 6.S191 Introduction to Deep Learning. All Rights Reserved.
# 
# Licensed under the MIT License. You may not use this file except in compliance
# with the License. Use and/or modification of this code outside of 6.S191 must
# reference:
#
# © MIT 6.S191: Introduction to Deep Learning
# http://introtodeeplearning.com
#

# Lab 1: Introduction à TensorFlow et à la génération de musique with RNNs

Dans ce laboratoire, vous vous familiariserez avec l'utilisation de TensorFlow et apprendrez comment l'utiliser pour résoudre des tâches d'apprentissage approfondi. Passez en revue le code et exécutez chaque cellule. En cours de route, vous rencontrerez plusieurs blocs A FAIRE - suivez les instructions pour les remplir avant de lancer ces cellules et de continuer.


# Part 1: Introduction à TensorFlow

## 0.1 Installer TensorFlow

TensorFlow est une bibliothèque de logiciels largement utilisée dans l'apprentissage machine. Nous apprendrons ici comment les calculs sont représentés et comment définir un simple réseau de neurones dans TensorFlow. Pour tous les laboratoires de 6.S191 2020, nous utiliserons la dernière version de TensorFlow, TensorFlow 2, qui offre une grande flexibilité et la possibilité d'exécuter impérativement des opérations, tout comme en Python. Vous remarquerez que TensorFlow 2 est assez similaire à Python dans sa syntaxe et son exécution impérative. Installons TensorFlow et quelques dépendances.


In [0]:
%tensorflow_version 2.x
import tensorflow as tf

# On charge et on importe le package MIT 6.S191
!pip install mitdeeplearning
import mitdeeplearning as mdl

import numpy as np
import matplotlib.pyplot as plt

## 1.1 Pourquoi TensorFlow se nomme TensorFlow ?

TensorFlow est appelé "TensorFlow" parce qu'il gère le flux (nœud/opération mathématique) des Tensors, qui sont des structures de données que l'on peut considérer comme des tableaux multidimensionnels. Les tenseurs sont représentés sous forme de tableaux à n dimensions de types de données de base tels qu'une chaîne ou un entier. Ils permettent de généraliser les vecteurs et les matrices à des dimensions plus élevées.

La "forme" d'un tenseur définit son nombre de dimensions et la taille de chaque dimension. Le "rang" d'un tenseur fournit le nombre de dimensions (n-dimensions) -- on peut aussi considérer qu'il s'agit de l'ordre ou du degré du tenseur.

Examinons d'abord les tenseurs 0-d, dont un scalaire est un exemple :

In [0]:
sport = tf.constant("Tennis", tf.string)
number = tf.constant(1.41421356237, tf.float64)

print("`sport` is a {}-d Tensor".format(tf.rank(sport).numpy()))
print("`number` is a {}-d Tensor".format(tf.rank(number).numpy()))

Les vecteurs et les listes peuvent être utilisés pour créer des tenseurs 1-d :

In [0]:
sports = tf.constant(["Tennis", "Basketball"], tf.string)
numbers = tf.constant([3.141592, 1.414213, 2.71821], tf.float64)

print("`sports` is a {}-d Tensor with shape: {}".format(tf.rank(sports).numpy(), tf.shape(sports)))
print("`numbers` is a {}-d Tensor with shape: {}".format(tf.rank(numbers).numpy(), tf.shape(numbers)))

Ensuite, nous envisageons de créer des tenseurs 2-d (c'est-à-dire des matrices) et des tenseurs de rang supérieur. Par exemple, dans les futurs laboratoires de traitement d'images et de vision par ordinateur, nous utiliserons des tenseurs 4-d. Ici, les dimensions correspondent au nombre d'images d'exemple dans notre lot, à la hauteur et à la largeur de l'image, ainsi qu'au nombre de canaux de couleur.

In [0]:
### Définir des tenseurs d'ordre supérieur ###

'''A FAIRE: Définir un tenseur 2-d'''
matrix = tf.constant([[1.0, 2.0, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0]]) # TODO
# matrix = # A FAIRE

assert isinstance(matrix, tf.Tensor), "matrix doit être un objet Tensor de tf"
assert tf.rank(matrix).numpy() == 2

In [0]:
'''A FAIRE: Définir un tenseur 4-d.'''
# Utiliser tf.zeros pour initialiser un tenseur 4d avec des zéros et d'une taille 10 x 256 x 256 x 3.  
#   On peut considérer qu'il s'agit de 10 images où chaque image est en RVB 256 x 256.
images = tf.zeros([10, 256, 256, 3]) # A FAIRE
# images = # A FAIRE

assert isinstance(images, tf.Tensor), "matrix must be a tf Tensor object"
assert tf.rank(images).numpy() == 4, "matrix must be of rank 4"
assert tf.shape(images).numpy().tolist() == [10, 256, 256, 3], "matrix is incorrect shape"

Comme vous l'avez vu, la "forme" d'un tenseur fournit le nombre d'éléments dans chaque dimension du tenseur. La "forme" est très utile, et nous l'utiliserons souvent. Vous pouvez également utiliser le découpage pour accéder aux sous-tenseurs d'un tenseur de rang supérieur :

In [0]:
row_vector = matrix[1]
column_vector = matrix[:,2]
scalar = matrix[1, 2]

print("`row_vector`: {}".format(row_vector.numpy()))
print("`column_vector`: {}".format(column_vector.numpy()))
print("`scalar`: {}".format(scalar.numpy()))

## 1.2 Calculs sur les tenseurs

Une façon pratique d'envisager et de visualiser les calculs dans TensorFlow est de les faire sous forme de graphiques. Nous pouvons définir ce graphique en termes de Tenseurs, qui contiennent des données, et les opérations mathématiques qui agissent sur ces Tenseurs dans un certain ordre. Prenons un exemple simple, et définissons ce calcul en utilisant TensorFlow :

![alt text](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab1/img/add-graph.png)

In [0]:
# Créer les noeuds dans le graphe et initialiser les valeurs
a = tf.constant(15)
b = tf.constant(61)

# On les ajoute !
c1 = tf.add(a,b)
c2 = a + b # TensorFlow surcharge l'opération "+" pour qu'elle puisse être utilisée sur les tenseurs
print(c1)
print(c2)

Remarquez comment nous avons créé un graphique de calcul composé d'opérations TensorFlow, et comment la sortie est un Tensor avec la valeur 76 -- nous venons de créer un graphique de calcul composé d'opérations, et il les a exécutées et nous a rendu le résultat.

Examinons maintenant un exemple un peu plus compliqué :

![alt text](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab1/img/computation-graph.png)

Ici, nous prenons deux entrées, a, b, et calculons une sortie e. Chaque noeud du graphique représente une opération qui prend une entrée, fait un calcul, et passe sa sortie à un autre noeud.

Définissons une fonction simple dans TensorFlow pour construire cette fonction de calcul :

In [0]:
### Définir des calculs de tenseurs ###

# Construire une fonction simple de calcul
def func(a,b):
  '''A FAIRE: Définir l'opération pour c, d, e (utiliser tf.add, tf.subtract, tf.multiply).'''
  c = tf.add(a, b)
  # c = # TODO
  d = tf.subtract(b, 1)
  # d = # TODO
  e = tf.multiply(c, d)
  # e = # TODO
  return e

Maintenant, nous pouvons appeler cette fonction pour exécuter le graphique de calcul étant donné certaines entrées "a,b" :

In [0]:
# Considérons les exemples de valeur pour a,b
a, b = 1.5, 2.5
# Exécuter le calcul
e_out = func(a,b)
print(e_out)


Remarquez que notre sortie est un tenseur dont la valeur est définie par la sortie du calcul, et que la sortie n'a pas de forme car il s'agit d'une valeur scalaire unique.

## 1.3 Réseaux de neurones dans TensorFlow
Nous pouvons également définir des réseaux de neurones dans TensorFlow. TensorFlow utilise une API de haut niveau appelée [Keras](https://www.tensorflow.org/guide/keras) qui fournit un cadre puissant et intuitif pour la construction et la formation de modèles d'apprentissage profond.

Considérons d'abord l'exemple d'un perceptron simple défini par une seule couche dense : $ y = \sigma(Wx + b)$, où $W$ représente une matrice de poids, $b$ est un biais, $x$ est l'entrée, $\sigma$ est la fonction d'activation sigmoïde, et $y$ est la sortie. Nous pouvons également visualiser cette opération à l'aide d'un graphique : 
 

![alt text](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab1/img/computation-graph-2.png)

Les tenseurs peuvent s'écouler à travers des types abstraits appelés [``Layers``] (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) -- les éléments constitutifs des réseaux de neurones. Les "couches" mettent en œuvre des opérations communes de réseaux de neurones, et sont utilisées pour mettre à jour les poids, calculer les pertes, et définir la connectivité entre les couches. Nous allons d'abord définir une "couche" pour mettre en œuvre le perceptron simple défini ci-dessus.


In [0]:
### Définir un calque de réseau ###

# n_output_nodes: nombre de noeuds de sortie
# input_shape: forme de l'entrée
# x: entrée de la couche

# La classe OurDenseLayer hérite de la classe tf.keras.layers.Layer
class OurDenseLayer(tf.keras.layers.Layer):
  # Un constructeur est défini passant le nombre de noeuds de sortie en paramètre
  def __init__(self, n_output_nodes):
    # On lance le constructeur de la classe mère tf.keras.layers.Layer
    super(OurDenseLayer, self).__init__()
    self.n_output_nodes = n_output_nodes

  def build(self, input_shape):
    # Rappel dans un tableau -1 est le dernier élément de la liste
    d = int(input_shape[-1])
    # Définir et intialiser les paramètres: une matrice de poids W et un biais b
    # Noter que l'initialisation des paramètres est aléatoire !
    self.W = self.add_weight("weight", shape=[d, self.n_output_nodes]) # noter la dimension
    self.b = self.add_weight("bias", shape=[1, self.n_output_nodes]) # noter la dimension

  def call(self, x):
    '''A FAIRE: définir l'opération pour z (utiliser tf.matmul)'''
    z = tf.matmul(x, self.W) + self.b # A FAIRE
    # z = # A FAIRE

    '''A FAIRE: définir l'opération pour la sortie (utiliser tf.sigmoid)'''
    y = tf.sigmoid(z) # A FAIRE
    # y = # A FAIRE
    return y

# Comme les paramètres des couches sont initialisés de manière aléatoire ?
# Nous allons définir une graine aléatoire pour la reproductibilité
tf.random.set_seed(1)
# On met en place une couche dense avec 3 noeuds de sortie 
layer = OurDenseLayer(3)
# On construit les tenseurs W et b à partir d'une entrée à 2 noeuds
# La tenseur de l'entrée est un tenseur d'une ligne avec 2 valeurs
# ce qui correspond à un tenseur de forme de forme (1, 2)
layer.build((1,2))
# On définit le tenseur d'entrée en définissant ses valeurs
x_input = tf.constant([[1,2.]], shape=(1,2))
y = layer.call(x_input)

# tester la sortie!
print(y.numpy())
mdl.lab1.test_custom_dense_layer_output(y)

De manière pratique, TensorFlow a défini un certain nombre de "couches" qui sont couramment utilisées dans les réseaux neuronaux, par exemple une Dense] (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense?version=stable). Maintenant, au lieu d'utiliser une seule "couche" pour définir notre simple réseau de neurones, nous allons utiliser le modèle [`Sequential de Keras et une seule couche Dense pour définir notre réseau. Avec l'API "séquentielle", vous pouvez facilement créer des réseaux de neurones en empilant des couches comme des blocs de construction.

In [0]:
### Définir un réseau de neurones en utilisant l'API Sequential ###

# Importer les packages utiles
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Définir le nombre de sortie
n_output_nodes = 3

# Premièrement définir le modèle 
model = Sequential()

'''A FAIRE: Définir une couche dense (totalement connectée) pour calculer z'''
# Rappel: les couches denses sont définies par les paramètres W et b!
# Vous pouvez lire plus sur l'initialisation de W et b dans la documentation de TF:) 
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense?version=stable
dense_layer = Dense(n_output_nodes, activation='sigmoid') # A FAIRE
# dense_layer = # A FAIRE

# Ajouter la couche dense au modèle
model.add(dense_layer)


C'est ça ! Nous avons défini notre modèle en utilisant l'API séquentielle. Maintenant, nous pouvons le tester en utilisant un exemple d'entrée :

In [0]:
# Tester le modèle avec un exemple d'entrée
x_input = tf.constant([[1,2.]], shape=(1,2))

'''A FAIRE: alimenter le modèle et prévoir le résultat!'''
model_output = model(x_input).numpy()
# model_output = # A FAIRE
print(model_output)

En plus de définir des modèles à l'aide de l'API "séquentielle", nous pouvons également définir des réseaux de neurones en sous-classant directement la classe [`Model`] (https://https://www.tensorflow.org/api_docs/python/tf/keras/Model?version=stable), qui regroupe des couches pour permettre l'apprentissage et l'inférence de modèles. La classe `Model` capture ce que nous appelons un "modèle" ou un "réseau". En utilisant la sous-classification, nous pouvons créer une classe pour notre modèle, puis définir le passage à travers le réseau en utilisant la fonction "call". Le sous-classement offre la flexibilité de définir des couches personnalisées, des boucles d'entraînement personnalisées, des fonctions d'activation personnalisées et des modèles personnalisés. Définissons maintenant le même réseau de neurones que ci-dessus en utilisant le sous-classement plutôt que le modèle "séquentiel".


In [0]:
### Définir un modèle en utilisant le sous-classement ###

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

class SubclassModel(tf.keras.Model):

  # Dans __init__, nous définissons les calques du modèle
  def __init__(self, n_output_nodes):
    super(SubclassModel, self).__init__()
    '''A FAIRE: Notre modèle consiste en un seul calque Dense. Définir ce calque.''' 
    self.dense_layer = Dense(n_output_nodes, activation='sigmoid') # TODO
    # self.dense_layer = '''A FAIRE: Calque Dense'''

  # Dans la fonction call, nous définissons le passage en avant du modèle.
  def call(self, inputs):
    return self.dense_layer(inputs)

Tout comme le modèle que nous avons construit en utilisant l'API "séquentielle", nous allons tester notre "SubclassModel" en utilisant un exemple d'entrée.



In [0]:
n_output_nodes = 3
model = SubclassModel(n_output_nodes)

x_input = tf.constant([[1,2.]], shape=(1,2))

print(model.call(x_input))

Il est important de noter que le sous-classement nous offre une grande flexibilité pour définir des modèles personnalisés. Par exemple, nous pouvons utiliser des arguments booléens dans la fonction "appel" pour spécifier différents comportements du réseau, par exemple différents comportements pendant la formation et l'inférence. Supposons que dans certains cas nous voulions que notre réseau produise simplement l'entrée, sans aucune perturbation. Nous définissons un argument booléen "isidentity" pour contrôler ce comportement :

In [0]:
### Définir un modèle en utilisant des sous-classes et en spécifiant un comportement personnalisé ###

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

class IdentityModel(tf.keras.Model):

  # Comme précédemment, dans __init__ nous définissons les couches du modèle
  # Puisque notre comportement souhaité implique le passage en avant, cette partie est inchangée
  def __init__(self, n_output_nodes):
    super(IdentityModel, self).__init__()
    self.dense_layer = tf.keras.layers.Dense(n_output_nodes, activation='sigmoid')

  '''A FAIRE: Mettre en œuvre le comportement selon lequel le réseau produit l'entrée, inchangée, sous le contrôle de l'argument isidentity.'''
  def call(self, inputs, isidentity=False):
    x = self.dense_layer(inputs)
    if isidentity: # A FAIRE
      return inputs # A FAIRE
    return x
  
  # def call(self, inputs, isidentity=False):
    # A FAIRE

Testons ce comportement :

In [0]:
n_output_nodes = 3
model = IdentityModel(n_output_nodes)

x_input = tf.constant([[1,2.]], shape=(1,2))
'''A FAIRE: passer l'entrée dans le modèle et appeler avec et sans l'option d'identité d'entrée.'''
out_activate = model.call(x_input) # A FAIRE
# out_activate = # A FAIRE
out_identity = model.call(x_input, isidentity=True) # A FAIRE
# out_identity = # A FAIRE

print("Network output with activation: {}; network identity output: {}".format(out_activate.numpy(), out_identity.numpy()))

Maintenant que nous avons appris à définir les "couches" ainsi que les réseaux neuronaux dans TensorFlow en utilisant à la fois les API "séquentielles" et de sous-classement, nous sommes prêts à nous intéresser à la manière de mettre en œuvre la formation des réseaux avec rétropropagation.

## 1.4 Dérivation automatique dans TensorFlow

[La dérivation automatique](https://en.wikipedia.org/wiki/Automatic_differentiation)
est l'une des parties les plus importantes de TensorFlow et constitue l'épine dorsale de l'apprentissage avec la
[backpropagation](https://en.wikipedia.org/wiki/Backpropagation). Nous utiliseraons le GradientTape de TensorFlow [`tf.GradientTape`](https://www.tensorflow.org/api_docs/python/tf/GradientTape?version=stable) pour retracer les opérations de calcul des gradients plus tard. 

Lorsqu'un passage en avant est effectué à travers le réseau, toutes les opérations de passage en avant sont enregistrées sur une "bande" ; ensuite, pour calculer le gradient, la bande est lue en arrière. Par défaut, la bande est rejetée après avoir été lue à l'envers ; cela signifie qu'une `tf.GradientTape` particulière ne peut calculer qu'un seul gradient, et les appels suivants provoquent une erreur d'exécution. Cependant, nous pouvons calculer plusieurs gradients sur le même calcul en créant une bande de gradients "persistante". 

Tout d'abord, nous examinerons comment calculer les gradients à l'aide de GradientTape et y accéder pour le calcul. Nous définissons la fonction simple $ y = x^2$ et calculons le gradient :


In [0]:
### Calcul du Gradient avec GradientTape ###

# y = x^2
# Exemple: x = 3.0
x = tf.Variable(3.0)

# Initialiser l'enregistrement du gradient
with tf.GradientTape() as tape:
  # Définir la fonction
  y = x * x
# Accéder au gradient -- dériver y par rapport à x
dy_dx = tape.gradient(y, x)

assert dy_dx.numpy() == 6.0

Dans l'entraînement des réseaux de neurones, nous utilisons la différenciation et la descente stochastique de gradient (SGD) pour optimiser une fonction de perte. Maintenant que nous avons une idée de la façon dont la "bande de gradient" peut être utilisée pour calculer et accéder aux dérivés, nous allons examiner un exemple où nous utilisons la différenciation automatique et la SGD pour trouver le minimum de $L=(x-x_f)^2$. Ici, $x_f$ est une variable pour une valeur souhaitée que nous essayons d'optimiser ; $L$ représente une perte que nous essayons de minimiser. Bien que nous puissions clairement résoudre ce problème de manière analytique ($x_{min}=x_f$), le fait de considérer comment nous pouvons le calculer en utilisant `GradientTape` nous prépare pour les futurs laboratoires où nous utiliserons la descente de gradient pour optimiser les pertes de réseaux neuronaux entiers.


In [0]:
### Minimisation de la fonction avec la différentiation automatique et le SGD ###

# Initialiser une variable aléatoire pour la valeur initiale de x
x = tf.Variable([tf.random.normal([1])])
print("Initializing x={}".format(x.numpy()))

learning_rate = 1e-2 # learning rate pour SGD
history = []
# Définir la valeur cible
x_f = 4

# Nous dirigerons le SGD pendant un certain nombre d'itérations. À chaque itération, nous calculons la perte,
#   calculons la dérivée de la perte par rapport à x, et effectuons la mise à jour du SGD..
for i in range(500):
  with tf.GradientTape() as tape:
    '''A FAIRE: définir la perte comme décrit ci-dessus'''
    loss = (x - x_f)**2 # "forward pass": enregistrer la perte courante sur la bande
    # loss = # A FAIRE

  # minimisation de la perte en utilisant l'enregistrement du gradient
  grad = tape.gradient(loss, x) # calculer la dérivée de la perte par rapport à x
  new_x = x - learning_rate*grad # mise à jour du sgd
  x.assign(new_x) # mettre à jour la valeur de x
  history.append(x.numpy()[0])

# Traçer l'évolution de x au cours de l'optimisation par rapport à x_f!
plt.plot(history)
plt.plot([0, 500],[x_f,x_f])
plt.legend(('Predicted', 'True'))
plt.xlabel('Iteration')
plt.ylabel('x value')

`GradientTape` provides an extremely flexible framework for automatic differentiation. In order to back propagate errors through a neural network, we track forward passes on the Tape, use this information to determine the gradients, and then use these gradients for optimization using SGD.